**Human-level control through Deep Reinforcement Learning**

https://deepmind.com/research/dqn/

https://deepmind.com/blog/deep-reinforcement-learning/

**Playing Atari with Deep Reinforcement Learning**

https://arxiv.org/abs/1312.5602
    
**Demystifying Deep Reinforcement Learning**

https://www.nervanasys.com/demystifying-deep-reinforcement-learning/

**Let’s make a DQN**

https://jaromiru.com/category/dqn/

**CartPole-v0**

https://gym.openai.com/envs/CartPole-v0

In [1]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.charts import HeatMap
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, HoverTool
from bokeh.palettes import Set1
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [2]:
from collections import deque, namedtuple
from datetime import timedelta
from time import time

import math
import random

import numpy as np
import tensorflow as tf

import gym

env = gym.make('CartPole-v0')

state_size = env.observation_space.shape[0] # 4
action_size = env.action_space.n # 2

[2017-05-24 15:25:23,719] Making new env: CartPole-v0


## Agent execution + Random Agent

In [3]:
RunState = namedtuple('RunState', ['i', 't', 'mean_t', 'min_t', 'max_t', 'R'])
Memento = namedtuple('Memento', ['state', 'action', 'reward', 'state_after'])

class NoMemory:
    def add(self, data):
        pass
    def sample(self, n):
        return []

def gym_run(agent_func,
            memory=NoMemory(),
            num_episodes=20,
            max_steps=500,
            report_buffer=1,
            report_func=lambda r: None):
    results = []
    start = time()
    n_t, mean_t, min_t, max_t = 0, 0, max_steps, 0
    terminal_state = None
    for i in range(1, num_episodes+1):
        done = False
        t = 0
        R = 0
        s = env.reset()
        while not done and t < max_steps:
            a = agent_func(s, memory)
            s_, r, done, _ = env.step(a)
            if done: s_ = terminal_state
            memory.add(Memento(s, a, r, s_))
            t += 1
            R += r
            s = s_
        
        n_t += 1
        mean_t = (n_t - 1) / n_t * mean_t + t / n_t
        min_t = min(min_t, t)
        max_t = max(max_t, t)
        results.append(RunState(i, t, mean_t, min_t, max_t, R))
        
        if report_buffer > 0 and i % report_buffer == 0:
            report_func(results)
            results.clear()

            duration = timedelta(seconds=time() - start)
            start = time()
            print('Episode {} last for {} steps, mean steps {:.2f} [{}, {}], time {}' \
                  .format(i, t, mean_t, min_t, max_t, duration))
            if report_buffer != 1:
                n_t, mean_t, min_t, max_t = 0, 0, max_steps, 0


terminal_state = np.zeros(state_size)
def make_state(s): return terminal_state if s is None else s

def random_agent(s, memory=None):
    return random.randint(0, action_size-1)

RunPlot = namedtuple('RunPlot', ['handle', 'steps_source', 'rewards_source'])

def plot_run():
    steps_data = ColumnDataSource(data=dict(episodes=[], t=[], mean_t=[], min_t=[], max_t=[]))
    rewards_data = ColumnDataSource(data=dict(episodes=[], r=[]))

    steps_hover = HoverTool(
        tooltips=[
            ('episode', '@episodes'),
            ('steps', '@t'),
            ('mean', '@mean_t'),
            ('min', '@min_t'),
            ('max', '@max_t'),
        ]
    )
    
    plot_steps = figure(width=475, height=300, tools=[steps_hover], logo=None, toolbar_location=None,
                        title='Steps', x_axis_label='episodes', y_axis_label='T',
                        x_range=DataRange1d(follow='end', follow_interval=100, range_padding=0))
    plot_steps.line('episodes', 't', color='yellow', source=steps_data)
    plot_steps.line('episodes', 'mean_t', color='green', source=steps_data)
    plot_steps.line('episodes', 'min_t', color='orange', source=steps_data)
    plot_steps.line('episodes', 'max_t', color='red', source=steps_data)

    rewards_hover = HoverTool(
        tooltips=[
            ('episode', '@episodes'),
            ('reward', '@r'),
        ]
    )
    
    plot_rewards = figure(width=475, height=300, tools=[rewards_hover], logo=None, toolbar_location=None,
                          title='Rewards', x_axis_label='episodes', y_axis_label='R',
                          x_range=DataRange1d(range_padding=0))
    plot_rewards.line('episodes', 'r', color='blue', source=rewards_data)
    handle = show(row(plot_steps, plot_rewards), notebook_handle=True)
    return RunPlot(handle, steps_data, rewards_data)

def report_plot(plot, results):
    episodes, t, mean_t, min_t, max_t, r = zip(*results)
    plot.steps_source.stream(dict(episodes=episodes, t=t, mean_t=mean_t, min_t=min_t, max_t=max_t), rollover=1000)
    plot.rewards_source.stream(dict(episodes=episodes, r=r))
    push_notebook(handle=plot.handle)

def plot_adapter(plot):
    return lambda r: report_plot(plot, r)

def run_agent(*args, **kargs):
    plot = plot_run()
    gym_run(*args, **kargs, report_func=plot_adapter(plot))

In [4]:
#%%script false

run_agent(random_agent)

Episode 1 last for 14 steps, mean steps 14.00 [14, 14], time 0:00:00.011661
Episode 2 last for 11 steps, mean steps 12.50 [11, 14], time 0:00:00.010313
Episode 3 last for 12 steps, mean steps 12.33 [11, 14], time 0:00:00.008904
Episode 4 last for 15 steps, mean steps 13.00 [11, 15], time 0:00:00.010647
Episode 5 last for 24 steps, mean steps 15.20 [11, 24], time 0:00:00.010936
Episode 6 last for 13 steps, mean steps 14.83 [11, 24], time 0:00:00.012918
Episode 7 last for 16 steps, mean steps 15.00 [11, 24], time 0:00:00.008954
Episode 8 last for 12 steps, mean steps 14.62 [11, 24], time 0:00:00.008265
Episode 9 last for 34 steps, mean steps 16.78 [11, 34], time 0:00:00.008851
Episode 10 last for 17 steps, mean steps 16.80 [11, 34], time 0:00:00.009940
Episode 11 last for 21 steps, mean steps 17.18 [11, 34], time 0:00:00.008532
Episode 12 last for 16 steps, mean steps 17.08 [11, 34], time 0:00:00.010260
Episode 13 last for 12 steps, mean steps 16.69 [11, 34], time 0:00:00.012158
Episode 

## Simple DQN Agent

* $\varepsilon$-greedy
* Uniform Experience Replay
* Single Q Network

(missing target network, error clipping)

**This algorithm is Unstable**

### $\varepsilon$-greedy

$\varepsilon = \varepsilon_{min} + (\varepsilon_{max} - \varepsilon_{min}) e^{-\lambda t}$

The $\lambda$ parameter controls the speed of decay. This way we start with a policy that explores greatly and behaves more and more greedily over time.

In [5]:
class EpsilonGreedy:
    
    def __init__(self, eps_min=0.01, eps_max=1.0, decay=0.001):
        self.eps_min = eps_min
        self.eps_max = eps_max
        self.decay = decay
        self.steps = 0

    def epsilon(self):
        return self.eps_min \
        + (self.eps_max - self.eps_min) \
        * math.exp(-self.decay * self.steps)
    
    def explore(self):
        return random.random() < self.epsilon()
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.steps += 1
        return self.explore()

### Uniform Memory (Experience Replay)

In [6]:
memory_size = 100000

class UniformMemory:
    
    def __init__(self, size=memory_size):
        self.data = deque(maxlen=size)

    def __del__(self):
        self.data.clear()

    def add(self, data):
        self.data.append(data)

    def sample(self, n):
        n = min(n, len(self.data))
        return random.sample(self.data, n)


### Q Network

The *Q Network* class encapsulates the neural network. Our problem is simple enough so we will use only one hidden layer of 64 neurons, with ReLU activation function. The final layer will consist of only two neurons, one for each available action. Their activation function will be linear. Remember that we are trying to approximate the Q function, which in essence can be of any real value. Therefore we can’t restrict the output from the network and the linear activation works well.

### Training

$Q(s, a) \xrightarrow{} r + \gamma max_{a'}{Q(s', a')}$

This formula means that for a sample $(s, r, a, s')$ we will update the network’s weights so that its output is closer to the target.

In [7]:
class QNetwork(namedtuple('QNetwork', ['x', 'y', 'y_hat', 'v_hat', 'pi_hat', 'loss', 'train_op'])):
    __slots__ = ()
    
    def train(self, session, X, Y):
        return session.run([self.train_op, self.loss], feed_dict={self.x: X, self.y: Y})[1]
    
    def predict_Q(self, session, X):
        return session.run(self.y_hat, feed_dict={self.x: X})
    
    def predict_V(self, session, X):
        return session.run(self.v_hat, feed_dict={self.x: X})
    
    def predict_action(self, session, s):
        X = s.reshape((1, state_size))
        return session.run(self.pi_hat, feed_dict={self.x: X})[0]

class DQNSimpleGraph:
    
    # self.graph
    # self.Q_net
    # self.init_op
    # self.input_size
    # self.output_size
    
    def __init__(self,
                 input_size=state_size,
                 output_size=action_size,
                 hidden_size=64,
                 learning_rate=0.00025):
        self.graph = tf.Graph()
        with self.graph.as_default():
            self.Q_net = self.q_network(input_size,
                                        output_size,
                                        hidden_size,
                                        learning_rate)
            self.init_op = tf.global_variables_initializer()
        
        self.input_size = input_size
        self.output_size = output_size
    
    def q_network(self, input_size, output_size, hidden_size, learning_rate):
        x = tf.placeholder(tf.float32, shape=[None, input_size])
        y = tf.placeholder(tf.float32, shape=[None, output_size])

        W_h = tf.Variable(tf.random_uniform([input_size, hidden_size], -1.0, 1.0))
        b_h = tf.Variable(tf.zeros([hidden_size]))

        hidden = tf.nn.relu(tf.matmul(x, W_h) + b_h)

        W_o = tf.Variable(tf.random_uniform([hidden_size, output_size], -1.0, 1.0))
        b_o = tf.Variable(tf.zeros([output_size]))

        y_hat = tf.matmul(hidden, W_o) + b_o

        v_hat = tf.reduce_max(y_hat, 1)
        pi_hat = tf.argmax(y_hat, 1)

        loss = tf.reduce_mean(tf.square(y_hat - y))
        opt = tf.train.RMSPropOptimizer(learning_rate)
        train_op = opt.minimize(loss)

        return QNetwork(x, y, y_hat, v_hat, pi_hat, loss, train_op)

class DQNSimple:
    
    def __init__(self,
                 q_graph = DQNSimpleGraph(),
                 batch_size=64,
                 gamma=0.99,
                 explore_policy=EpsilonGreedy()):
        self.q_graph = q_graph
        self.batch_size = batch_size
        self.gamma = gamma
        self.explore_policy = explore_policy

        self.session = tf.Session(graph=q_graph.graph)
        self.session.run(q_graph.init_op)
    
    def __del__(self):
        self.session.close()
    
    # Q(s, a) -> r + gamma * max_a' Q(s', a')
    def update_q(self, memory):
        replay_sample = memory.sample(self.batch_size)
        if not replay_sample:
            return
        session = self.session
        Q_net = self.q_graph.Q_net
        gamma = self.gamma
        
        states = np.array([t.state for t in replay_sample])
        states_ = np.array([make_state(t.state_after) for t in replay_sample])
        
        Q = Q_net.predict_Q(session, states)
        max_Q = Q_net.predict_V(session, states_)
        
        n = len(replay_sample)
        X = np.zeros((n, self.q_graph.input_size))
        Y = np.zeros((n, self.q_graph.output_size))
        for i, t in enumerate(replay_sample):
            s, a, r, s_ = t
            X[i] = s
            q = Q[i]
            if s_ is None:
                q[a] = r
            else:
                q[a] = r + gamma * max_Q[i]
            Y[i] = q
        loss_value = Q_net.train(session, X, Y)
        #print(loss_value)
    
    def Q(self, X):
        return self.q_graph.Q_net.predict_Q(self.session, X)
    
    def explore(self):
        return next(self.explore_policy)
    
    def action(self, s):
        return self.q_graph.Q_net.predict_action(self.session, s)
    
    def __call__(self, s, memory):
        self.update_q(memory)
        if self.explore():
            return random_agent(s)
        return self.action(s)

In [8]:
%%time

dqn_simple = DQNSimple()
memory = UniformMemory()

run_agent(dqn_simple, memory, num_episodes=2000, report_buffer=100)

Episode 100 last for 14 steps, mean steps 12.80 [8, 45], time 0:00:02.628597
Episode 200 last for 10 steps, mean steps 10.18 [8, 15], time 0:00:02.142136
Episode 300 last for 8 steps, mean steps 10.22 [8, 15], time 0:00:02.187213
Episode 400 last for 119 steps, mean steps 53.68 [9, 200], time 0:00:11.488874
Episode 500 last for 157 steps, mean steps 119.77 [47, 200], time 0:00:25.596788
Episode 600 last for 156 steps, mean steps 161.54 [130, 200], time 0:00:34.548125
Episode 700 last for 153 steps, mean steps 168.00 [135, 200], time 0:00:36.956175
Episode 800 last for 159 steps, mean steps 173.83 [138, 200], time 0:00:38.936207
Episode 900 last for 200 steps, mean steps 179.47 [140, 200], time 0:00:41.624876
Episode 1000 last for 148 steps, mean steps 180.48 [141, 200], time 0:00:42.803503
Episode 1100 last for 188 steps, mean steps 190.45 [139, 200], time 0:00:45.282899
Episode 1200 last for 187 steps, mean steps 197.43 [163, 200], time 0:00:47.362964
Episode 1300 last for 200 steps, 

### Q values sampling

In [9]:
Q_params = []

def q_collector(s, memory=None):
    a = dqn_simple.action(s)
    Q_params.append((s, a))
    return a

gym_run(q_collector, num_episodes=1, report_buffer=0)

q_index = [0, len(Q_params) // 4 - 1, len(Q_params) // 2 - 1, 3 * len(Q_params) // 4 - 1, -1]
s_sample, a_sample = zip(*[Q_params[i] for i in q_index])

X_sample = np.array(s_sample).reshape(len(s_sample), state_size)

a_index = [(i,a) for i, a in enumerate(a_sample)]
q_sample = np.random.rand(len(a_sample), action_size)

Q_values = []
step_count = 0
step_sample = 100

def q_sampler(s, memory):
    a = dqn_simple(s, memory)
    global step_count
    if step_count % step_sample == 0:
        q_sample = dqn_simple.Q(X_sample)
        q_sample = [q_sample[k] for k in a_index]
        Q_values.append(q_sample)
    step_count += 1
    return a

In [10]:
%%time

run_agent(q_sampler, memory, num_episodes=1000, report_buffer=100)

Episode 100 last for 14 steps, mean steps 12.17 [10, 16], time 0:00:04.082650
Episode 200 last for 21 steps, mean steps 60.48 [13, 200], time 0:00:19.319860
Episode 300 last for 14 steps, mean steps 16.12 [12, 22], time 0:00:03.958933
Episode 400 last for 10 steps, mean steps 12.04 [9, 16], time 0:00:02.960998
Episode 500 last for 14 steps, mean steps 12.42 [10, 14], time 0:00:03.051436
Episode 600 last for 200 steps, mean steps 24.23 [12, 200], time 0:00:05.884342
Episode 700 last for 200 steps, mean steps 200.00 [200, 200], time 0:00:52.034206
Episode 800 last for 200 steps, mean steps 200.00 [200, 200], time 0:00:50.173088
Episode 900 last for 200 steps, mean steps 200.00 [200, 200], time 0:00:48.837889
Episode 1000 last for 200 steps, mean steps 200.00 [200, 200], time 0:00:48.793021
CPU times: user 5min 11s, sys: 18.5 s, total: 5min 30s
Wall time: 3min 59s


In [11]:
q_samples = tuple(zip(*Q_values))
steps = range(1, len(Q_values) + 1)

plot = figure(width=900, tools='hover', logo=None,
              title='Q samples', x_axis_label='steps', y_axis_label='Q(s,a)')

for i, q_sample in enumerate(q_samples):
    plot.line(steps, q_sample, legend='Sample {}'.format(i+1), color=Set1[len(q_samples)][i])

show(plot)

/home/cavani/Workspace/tensorflow-jupyter/software/tensorflow-cpu/lib/python3.5/site-packages/bokeh/core/json_encoder.py:73: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  if pd and isinstance(obj, pd.tslib.Timestamp):


In [12]:
del memory
del dqn_simple

## DQN

* Target Network
* Error clipping

**Target Network**

$Q(s, a) \xrightarrow{} r + \gamma max_{a'}{\tilde{Q}(s', a')}$

After severals steps, the target network $\tilde{Q}$ is updated, just by copying the weights from the current network $Q$.

**Error clipping**

The loss function is directly used in the backward propagation algorithm and large errors cause large changes to the network.

Huber loss.

https://en.wikipedia.org/wiki/Huber_loss

In [13]:
class DQNGraph:
    
    # self.graph
    # self.Q_net
    # self.Q_target
    # self.update_target_op
    # self.init_op
    # self.input_size
    # self.output_size
    
    def __init__(self,
                 input_size=state_size,
                 output_size=action_size,
                 hidden_size=64,
                 learning_rate=0.00025):
        self.graph = tf.Graph()
        with self.graph.as_default():
            common_variables = ['W_h:0', 'b_h:0', 'W_o:0', 'b_o:0']
            with tf.variable_scope('q_network'):
                self.Q_net = self.q_network(input_size, output_size, hidden_size, learning_rate)
            with tf.variable_scope('q_target'):
                self.Q_target = self.q_network(input_size, output_size, hidden_size, learning_rate, False)

            source_variables = [self.graph.get_tensor_by_name('q_network/' + name) for name in common_variables]
            target_variables = [self.graph.get_tensor_by_name('q_target/' + name) for name in common_variables]
            assign_target = [tf.assign(v_target, v_source)
                             for v_target, v_source in zip(target_variables, source_variables)] 
            self.update_target_op = tf.group(*assign_target)

            self.init_op = tf.global_variables_initializer()

        self.input_size = input_size
        self.output_size = output_size
    
    def q_network(self, input_size, output_size, hidden_size, learning_rate, trainable=True):
        x = tf.placeholder(tf.float32, shape=[None, input_size])
        y = tf.placeholder(tf.float32, shape=[None, action_size])

        W_h = tf.Variable(tf.random_uniform([input_size, hidden_size], -1.0, 1.0), name='W_h')
        b_h = tf.Variable(tf.zeros([hidden_size]), name='b_h')

        hidden = tf.nn.relu(tf.matmul(x, W_h) + b_h)

        W_o = tf.Variable(tf.random_uniform([hidden_size, output_size], -1.0, 1.0), name='W_o')
        b_o = tf.Variable(tf.zeros([output_size]), name='b_o')

        y_hat = tf.matmul(hidden, W_o) + b_o

        v_hat = tf.reduce_max(y_hat, 1)
        pi_hat = tf.argmax(y_hat, 1)

        loss, train_op = None, None
        if trainable:
            loss = tf.reduce_mean(tf.sqrt(tf.square(y_hat - y) + 1) - 1)
            opt = tf.train.RMSPropOptimizer(learning_rate)
            train_op = opt.minimize(loss)

        return QNetwork(x, y, y_hat, v_hat, pi_hat, loss, train_op)

class DQN:
    
    def __init__(self,
                 q_graph=DQNGraph(),
                 batch_size=64,
                 gamma=0.99,
                 explore_policy=EpsilonGreedy()):
        self.q_graph = q_graph
        self.batch_size = batch_size
        self.gamma = gamma
        self.explore_policy = explore_policy

        self.session = tf.Session(graph=q_graph.graph)
        self.session.run(q_graph.init_op)
        self.update_target()

        
        self.steps_count = 0
    
    def __del__(self):
        self.session.close()

    def update_q(self, memory):
        replay_sample = memory.sample(self.batch_size)
        if not replay_sample:
            return
        
        session = self.session
        Q_net = self.q_graph.Q_net
        Q_target = self.q_graph.Q_target
        gamma = self.gamma
        
        states = np.array([t.state for t in replay_sample])
        states_ = np.array([make_state(t.state_after) for t in replay_sample])

        Q = Q_net.predict_Q(session, states)
        max_Q = Q_target.predict_V(session, states_)
        
        n = len(replay_sample)
        X = np.zeros((n, self.q_graph.input_size))
        Y = np.zeros((n, self.q_graph.output_size))
        for i, t in enumerate(replay_sample):
            s, a, r, s_ = t
            X[i] = s
            q = Q[i]
            if s_ is None:
                q[a] = r
            else:
                q[a] = r + gamma * max_Q[i]
            Y[i] = q
        loss_value = Q_net.train(session, X, Y)
        #print(loss_value)

    def Q(self, X):
        return self.q_graph.Q_net.predict_Q(self.session, X)
    
    def explore(self):
        return next(self.explore_policy)
    
    def action(self, s):
        return self.q_graph.Q_net.predict_action(self.session, s)
    
    def steps_completed(self, n=1000):
        self.steps_count += 1
        return self.steps_count % n == 0

    def update_target(self):
        self.session.run(self.q_graph.update_target_op)

    def __call__(self, s, memory):
        if self.steps_completed(1000):
            self.update_target()
        self.update_q(memory)
        if self.explore():
            return random_agent(s)
        return self.action(s)

In [14]:
%%time
#%%script false

memory = UniformMemory()

print('Fill Memory with Random\n')
gym_run(random_agent, memory, num_episodes=10000, report_buffer=1000)

dqn_agent = DQN()

print('\nDQN')
run_agent(dqn_agent, memory, num_episodes=2000, report_buffer=100)

del memory
del dqn_agent

Fill Memory with Random

Episode 1000 last for 18 steps, mean steps 22.45 [8, 113], time 0:00:00.343974
Episode 2000 last for 17 steps, mean steps 21.41 [8, 94], time 0:00:00.316056
Episode 3000 last for 15 steps, mean steps 21.69 [8, 84], time 0:00:00.364078
Episode 4000 last for 13 steps, mean steps 22.30 [8, 88], time 0:00:00.309053
Episode 5000 last for 14 steps, mean steps 22.00 [8, 129], time 0:00:00.301845
Episode 6000 last for 15 steps, mean steps 21.88 [8, 98], time 0:00:00.306075
Episode 7000 last for 21 steps, mean steps 21.97 [8, 88], time 0:00:00.316258
Episode 8000 last for 24 steps, mean steps 22.78 [8, 135], time 0:00:00.316416
Episode 9000 last for 20 steps, mean steps 22.34 [8, 96], time 0:00:00.322221
Episode 10000 last for 11 steps, mean steps 22.17 [8, 81], time 0:00:00.312597

DQN


Episode 100 last for 81 steps, mean steps 42.61 [9, 128], time 0:00:10.401086
Episode 200 last for 176 steps, mean steps 193.18 [70, 200], time 0:00:46.861420
Episode 300 last for 164 steps, mean steps 175.11 [131, 200], time 0:00:42.994957
Episode 400 last for 200 steps, mean steps 182.44 [142, 200], time 0:00:44.472129
Episode 500 last for 178 steps, mean steps 173.17 [108, 200], time 0:00:41.840597
Episode 600 last for 133 steps, mean steps 125.83 [108, 161], time 0:00:30.484045
Episode 700 last for 84 steps, mean steps 111.49 [82, 141], time 0:00:27.456491
Episode 800 last for 70 steps, mean steps 79.66 [68, 98], time 0:00:19.800673
Episode 900 last for 82 steps, mean steps 76.04 [66, 91], time 0:00:18.605078
Episode 1000 last for 84 steps, mean steps 84.64 [74, 100], time 0:00:20.812353
Episode 1100 last for 81 steps, mean steps 82.47 [72, 92], time 0:00:20.423220
Episode 1200 last for 83 steps, mean steps 79.79 [70, 93], time 0:00:19.504476
Episode 1300 last for 66 steps, mean st

## DDQN

$Q(s, a) \xrightarrow{} r + \gamma \tilde{Q}(s', argmax_{a'}{Q(s', a')})$

In [15]:
class DDQN(DQN):
    
    def update_q(self, memory):
        replay_sample = memory.sample(self.batch_size)
        if not replay_sample:
            return

        session = self.session
        Q_net = self.q_graph.Q_net
        Q_target = self.q_graph.Q_target
        gamma = self.gamma
        
        states = np.array([t.state for t in replay_sample])
        states_ = np.array([make_state(t.state_after) for t in replay_sample])
        
        Q1 = Q_net.predict_Q(session, states)
        Q1_ = Q_net.predict_Q(session, states_) 
        max_a = np.argmax(Q1_, axis=1)
        Q2 = Q_target.predict_Q(session, states_)
        
        n = len(replay_sample)
        X = np.zeros((n, self.q_graph.input_size))
        Y = np.zeros((n, self.q_graph.output_size))
        for i, t in enumerate(replay_sample):
            s, a, r, s_ = t
            X[i] = s
            q = Q1[i]
            if s_ is None:
                q[a] = r
            else:
                q[a] = r + gamma * Q2[i][max_a[i]]
            Y[i] = q
        loss_value = Q_net.train(session, X, Y)
        #print(loss_value)

In [16]:
%%time
#%%script false

memory = UniformMemory()

print('Fill Memory with Random\n')
gym_run(random_agent, memory, num_episodes=10000, report_buffer=1000)

ddqn_agent = DDQN()

print('\nDDQN')
run_agent(ddqn_agent, memory, num_episodes=2000, report_buffer=100)

del memory
del ddqn_agent

Fill Memory with Random

Episode 1000 last for 36 steps, mean steps 21.94 [8, 107], time 0:00:00.411423
Episode 2000 last for 29 steps, mean steps 22.12 [8, 95], time 0:00:00.307729
Episode 3000 last for 42 steps, mean steps 22.31 [8, 90], time 0:00:00.299905
Episode 4000 last for 54 steps, mean steps 22.19 [8, 85], time 0:00:00.324023
Episode 5000 last for 17 steps, mean steps 22.14 [8, 102], time 0:00:00.323113
Episode 6000 last for 20 steps, mean steps 21.07 [8, 109], time 0:00:00.299098
Episode 7000 last for 18 steps, mean steps 21.68 [8, 80], time 0:00:00.306639
Episode 8000 last for 36 steps, mean steps 22.35 [8, 111], time 0:00:00.326668
Episode 9000 last for 33 steps, mean steps 22.21 [8, 106], time 0:00:00.378963
Episode 10000 last for 31 steps, mean steps 21.85 [8, 89], time 0:00:00.309759

DDQN


Episode 100 last for 8 steps, mean steps 12.83 [8, 43], time 0:00:03.872493
Episode 200 last for 9 steps, mean steps 9.34 [8, 12], time 0:00:02.909646
Episode 300 last for 9 steps, mean steps 9.38 [8, 11], time 0:00:02.751733
Episode 400 last for 10 steps, mean steps 9.46 [8, 12], time 0:00:02.939962
Episode 500 last for 10 steps, mean steps 9.45 [8, 11], time 0:00:02.982568
Episode 600 last for 9 steps, mean steps 9.36 [8, 12], time 0:00:02.813812
Episode 700 last for 9 steps, mean steps 9.75 [8, 16], time 0:00:02.850913
Episode 800 last for 137 steps, mean steps 40.55 [9, 200], time 0:00:11.577903
Episode 900 last for 119 steps, mean steps 126.35 [103, 159], time 0:00:35.946718
Episode 1000 last for 174 steps, mean steps 155.55 [115, 200], time 0:00:44.692375
Episode 1100 last for 200 steps, mean steps 189.95 [149, 200], time 0:00:54.181831
Episode 1200 last for 96 steps, mean steps 130.18 [66, 200], time 0:00:36.162510
Episode 1300 last for 93 steps, mean steps 76.07 [19, 103], time

## Priorized Experience Replay

**Priority**

$p = (error + \epsilon)^\alpha$

* $\epsilon$ small positive constant
* $\alpha \in [0, 1]$ difference between high an low error

**DDQN Error**

$error = |Q(s, a) - T(s')|$

$T(s') = r + \gamma \tilde{Q}(s', argmax_{a'}{Q(s', a')})$

In [17]:
# Efficient sampling using Sum Tree

class WeightedTree:
    
    def __init__(self, min_size):
        data_size = 2 ** math.ceil(np.log2(min_size))
        self.data_size = data_size
        self.data_level = data_size - 1
        self.data_index = 0 # 0 .. data_size-1
        self.tree = np.zeros(2 * data_size - 1)
        self.data = [None] * data_size
    
    def __str__(self):
        return str(self.tree)
    
    def max_range(self):
        return self.tree[0]

    def add(self, data, weight):
        self.set_weight(self.data_index, weight)
        self.data[self.data_index] = data
        self.data_index = (self.data_index + 1) % self.data_size

    def set_weight(self, data_index, weight):
        i = self.data_level + data_index
        delta = weight - self.tree[i]
        self.tree[i] = weight
        while i:
            i = (i - 1) // 2
            self.tree[i] += delta

    def _select(self, i, range_value):
        while i < self.data_level:
            left = 2 * i + 1
            right = left + 1
            if range_value <= self.tree[left]:
                i = left
            else:
                i, range_value = right, range_value - self.tree[left]
        return i

    def select(self, range_value):
        i = self._select(0, range_value)
        data_index = i - self.data_level
        return (data_index, self.data[data_index], self.tree[i])


priority_eps = 0.01
priority_alpha = 0.6

class PriorizedMemory:
    
    def __init__(self,
                 error_func=lambda m: 0,
                 min_size=memory_size,
                 priority_eps=priority_eps,
                 priority_alpha=priority_alpha):
        self.data = WeightedTree(min_size)
        self.priority_eps = priority_eps
        self.priority_alpha = priority_alpha
        self._error = error_func
    
    def _priority(self, error):
        return (error + self.priority_eps) ** self.priority_alpha
    
    def add(self, memento):
        err = self._error(memento)
        p = self._priority(err)
        self.data.add(memento, p)
    
    def sample(self, n):
        sample = []

        range_size = self.data.max_range() / n
        for i in range(n):
            start = i * range_size
            stop = (i + 1) * range_size

            range_value = random.uniform(start, stop)
            k, memento, _ = self.data.select(range_value)
            if memento is not None:
                sample.append((k, memento))

        return sample

    def update(self, k, error):
        p = self._priority(error)
        self.data.set_weight(k, p)

In [18]:
class DDQN_PER(DQN):
    
    def error(self, memento):
        _, _, err = self.train_data([(-1, memento)])
        return err[0][1]
    
    def train_data(self, sample):
        states = np.array([t.state for _, t in sample])
        states_ = np.array([make_state(t.state_after) for _, t in sample])
        
        session = self.session
        Q_net = self.q_graph.Q_net
        Q_target = self.q_graph.Q_target
        gamma = self.gamma

        Q1 = Q_net.predict_Q(session, states)
        Q1_ = Q_net.predict_Q(session, states_) 
        max_a = np.argmax(Q1_, axis=1)
        Q2 = Q_target.predict_Q(session, states_)
        
        n = len(sample)
        X = np.zeros((n, self.q_graph.input_size))
        Y = np.zeros((n, self.q_graph.output_size))
        errors = [None] * n
        for i, (k, t) in enumerate(sample):
            s, a, r, s_ = t
            X[i] = s
            q = Q1[i]
            q_a = q[a]
            if s_ is None:
                q[a] = r
            else:
                q[a] = r + gamma * Q2[i][max_a[i]]
            Y[i] = q
            errors[i] = (k, abs(q_a - q[a]))
        
        return X, Y, errors
    
    def update_q(self, memory):
        replay_sample = memory.sample(self.batch_size)
        if not replay_sample:
            return

        X, Y, errors = self.train_data(replay_sample)
        
        for k, err in errors:
            memory.update(k, err)

        session = self.session
        Q_net = self.q_graph.Q_net

        loss_value = Q_net.train(session, X, Y)
        #print(loss_value)

In [19]:
%%time
#%%script false

ddqn_per_agent = DDQN_PER()

memory = PriorizedMemory(ddqn_per_agent.error)

print('\nDDQN+PER')
run_agent(ddqn_per_agent, memory, num_episodes=2000, report_buffer=100)

del memory
del ddqn_per_agent


DDQN+PER


Episode 100 last for 10 steps, mean steps 9.41 [8, 11], time 0:00:04.788127
Episode 200 last for 10 steps, mean steps 9.33 [8, 11], time 0:00:04.741420
Episode 300 last for 9 steps, mean steps 9.52 [8, 11], time 0:00:04.896447
Episode 400 last for 9 steps, mean steps 9.42 [8, 11], time 0:00:04.801246
Episode 500 last for 9 steps, mean steps 9.50 [8, 11], time 0:00:04.845025
Episode 600 last for 10 steps, mean steps 9.35 [8, 12], time 0:00:04.803417
Episode 700 last for 10 steps, mean steps 12.83 [8, 82], time 0:00:06.728010
Episode 800 last for 10 steps, mean steps 9.71 [8, 22], time 0:00:05.027640
Episode 900 last for 9 steps, mean steps 9.50 [8, 12], time 0:00:04.897053
Episode 1000 last for 23 steps, mean steps 11.77 [8, 36], time 0:00:06.039988
Episode 1100 last for 120 steps, mean steps 48.50 [10, 200], time 0:00:24.333208
Episode 1200 last for 143 steps, mean steps 99.01 [33, 200], time 0:00:49.578704
Episode 1300 last for 162 steps, mean steps 179.08 [136, 200], time 0:01:30.229